In [1]:
import GetOldTweets3 as got
from datetime import date, timedelta
import tweepy
import nltk
import re
import string
from nltk.corpus import stopwords
import contractions
import time
import math
from collections import Counter
import pickle
import pandas as pd
import numpy as np
import math

In [2]:
punct = string.punctuation
english_stopwords = stopwords.words('english')

teams_dict={"mi":"mumbai indians","csk":"chennai super kings","rcb":"royal challengers bangalore","kxip":"kings xi punjab",
            "kkr":"kolkata knight riders","dc":"delhi capitals","srh":"sunrisers hyderabad","rr":"rajasthan royals"}

# connection to Twitter API using tweepy

consumer_key = "CtmrTSUsnIRHKSgj4ktqK4NKb"
consumer_secret = "1MNfKmptTsrJcq7WMYxOoVwEgFCK5DNMQeC43IZDkPucPObCnZ"
access_key = "1231240089600057344-rqYnkeVvMaB1XwfvrERfI7aF38r69L"
access_secret = "KmRfAoexHxfkHKJaC3hwB0sgtfocyz58IqdzYiWeaXSGO"
# Authorization to consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

# Access to user's access key and access secret
auth.set_access_token(access_key, access_secret)

# Calling api
api = tweepy.API(auth)

In [3]:
# returns Twitter name based on twitter handle
def get_username(username):
    # 200 tweets to be extracted
    # number_of_tweets=1
    try:
        tweets = api.user_timeline(screen_name=username)

        # tweets_for_csv = [tweet.text for tweet in tweets]  # CSV file created
        for tweet in tweets:
            # print(tweet)
            return tweet._json['user']['name']
    except tweepy.error.TweepError:
        return None

In [4]:
date_list = []

# list of authoritative sources
source_list = ["@IPL", "@ESPNcricinfo", "@cricbuzz", "@Cricketracker", "@circleofcricket"]
#source_list = ["@ESPNcricinfo", "@cricbuzz", "@Cricketracker", "@circleofcricket"]
# source_list=["@circleofcricket"]

# hashtag to be filtered
hashtags = ["IPL"]

# max number of tweets per source
num_tweets = 20

# tweet range
start_date = date(2019, 3, 23)
end_date = date(2019, 4, 16)

# generate days between start and end date
delta = end_date - start_date
for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    # print(day)
    # print(day+timedelta(days=1))
    date_list.append([day, day + timedelta(days=1)])
date_list

[[datetime.date(2019, 3, 23), datetime.date(2019, 3, 24)],
 [datetime.date(2019, 3, 24), datetime.date(2019, 3, 25)],
 [datetime.date(2019, 3, 25), datetime.date(2019, 3, 26)],
 [datetime.date(2019, 3, 26), datetime.date(2019, 3, 27)],
 [datetime.date(2019, 3, 27), datetime.date(2019, 3, 28)],
 [datetime.date(2019, 3, 28), datetime.date(2019, 3, 29)],
 [datetime.date(2019, 3, 29), datetime.date(2019, 3, 30)],
 [datetime.date(2019, 3, 30), datetime.date(2019, 3, 31)],
 [datetime.date(2019, 3, 31), datetime.date(2019, 4, 1)],
 [datetime.date(2019, 4, 1), datetime.date(2019, 4, 2)],
 [datetime.date(2019, 4, 2), datetime.date(2019, 4, 3)],
 [datetime.date(2019, 4, 3), datetime.date(2019, 4, 4)],
 [datetime.date(2019, 4, 4), datetime.date(2019, 4, 5)],
 [datetime.date(2019, 4, 5), datetime.date(2019, 4, 6)],
 [datetime.date(2019, 4, 6), datetime.date(2019, 4, 7)],
 [datetime.date(2019, 4, 7), datetime.date(2019, 4, 8)],
 [datetime.date(2019, 4, 8), datetime.date(2019, 4, 9)],
 [datetime.dat

In [5]:
team_hashtag_dict={}
#generate team1vteam2 for matching hashtags such as #mivcsk
for k1 in list(teams_dict):
    for k2 in list(teams_dict):
        if k1!=k2:
            key="{}v{}".format(k1, k2)
            value="{} {}".format(teams_dict[k1],teams_dict[k2])
            team_hashtag_dict[key]=value
team_hashtag_dict

{'mivcsk': 'mumbai indians chennai super kings',
 'mivrcb': 'mumbai indians royal challengers bangalore',
 'mivkxip': 'mumbai indians kings xi punjab',
 'mivkkr': 'mumbai indians kolkata knight riders',
 'mivdc': 'mumbai indians delhi capitals',
 'mivsrh': 'mumbai indians sunrisers hyderabad',
 'mivrr': 'mumbai indians rajasthan royals',
 'cskvmi': 'chennai super kings mumbai indians',
 'cskvrcb': 'chennai super kings royal challengers bangalore',
 'cskvkxip': 'chennai super kings kings xi punjab',
 'cskvkkr': 'chennai super kings kolkata knight riders',
 'cskvdc': 'chennai super kings delhi capitals',
 'cskvsrh': 'chennai super kings sunrisers hyderabad',
 'cskvrr': 'chennai super kings rajasthan royals',
 'rcbvmi': 'royal challengers bangalore mumbai indians',
 'rcbvcsk': 'royal challengers bangalore chennai super kings',
 'rcbvkxip': 'royal challengers bangalore kings xi punjab',
 'rcbvkkr': 'royal challengers bangalore kolkata knight riders',
 'rcbvdc': 'royal challengers bangalore

In [6]:
tweet_dict = {}
tweet_tokens_dict={}
tokens_df_dict = {}
final_token_list=[]

tweetID = 0

for d in date_list:
    for source in source_list:
        max_tweet_score=0
        max_like_score=0
        startID=tweetID
        for hashtag in hashtags:
            print(d[0])
            print(source)
            print(hashtag)

            start = time.time()

            # set tweet criteria
            tweetCriteria = got.manager.TweetCriteria().setQuerySearch("".format(hashtag)) \
                .setSince("{}".format(d[0])) \
                .setUntil("{}".format(d[1])) \
                .setLang("en") \
                .setUsername("{}".format(source)) \
                .setTopTweets(num_tweets) \
                .setMaxTweets(num_tweets)

            # fetch tweets based on criteria
            tweets = got.manager.TweetManager.getTweets(tweetCriteria)

            # end = time.time()
            # print("{} sec".format(end - start))
            # start = time.time()

            # counter=0

            # for tweet in tweets:
            #    if hashtag in tweet.text:
            #        counter+=1

            # print(counter)

            for tweet in tweets:
                if hashtag in tweet.text:
                    final_tokens = []

                    # remove url from tweet text
                    tweet_text = re.sub(r"(http|https)\S+", "", tweet.text)
                    #tweet_text=re.sub(r'\B#\w*[a-zA-Z]+\w*',"",tweet_text)

                    # Fix contractions
                    tweet_text = contractions.fix(tweet_text)
                    #print(tweet_text)

                    if tweet.retweets > 0:
                        # print("---------------")
                        # print(tweet.username)
                        # print(tweet.date)

                        # tokenize
                        token_list = nltk.word_tokenize(tweet_text)
                        i = 0
                        while i < len(token_list):
                            if (token_list[i] == '@'):  # replace twitter handle with screen name
                                temp = token_list[i] + token_list[i + 1]
                                # print(temp)
                                if get_username(temp) is not None:
                                    # print(get_username(temp))
                                    temp_list = nltk.word_tokenize(get_username(temp))
                                    for t in temp_list:
                                        final_tokens.append(t.lower())
                                i = i + 2
                            # elif token_list[i].isnumeric(): #ignore digits from token list
                            #    temp = num2words(token_list[i])
                            # final_tokens.append(temp)
                            #    i=i+1
                            else:
                                flag = 1
                                # remove punctuation from each string
                                str_temp=token_list[i].translate(str.maketrans('', '', punct))
                                # print(str_temp)

                                # convert to lower case
                                str_temp = str_temp.lower()

                                # replace team abbreviations with team name
                                for team, team_name in teams_dict.items():
                                    if str_temp == team:
                                        temp_list = nltk.word_tokenize(team_name)
                                        # print(temp_list)
                                        for t in temp_list:
                                            final_tokens.append(t)
                                        flag = 0
                                if flag == 0:
                                    i = i + 1
                                    continue
                                    
                                for team, team_name in team_hashtag_dict.items():
                                    if str_temp == team:
                                        temp_list = nltk.word_tokenize(team_name)
                                        # print(temp_list)
                                        for t in temp_list:
                                            final_tokens.append(t)
                                        flag = 0
                                if flag == 0:
                                    i = i + 1
                                    continue
                                    
                                if str_temp.isnumeric() or str_temp in punct:
                                    #print(str_temp)
                                    i=i+1
                                    continue

                                # stop word and punctuation tokens removal
                                if str_temp not in english_stopwords:
                                    # str_temp = str_temp.replace("'", "")
                                    final_tokens.append(str_temp)
                            i = i + 1
                        # print(tweet.text)
                        # print(tweet.favorites)
                        # print(tweet.retweets)
                        # print("---------------")
                        
                        max_tweet_score=max(max_tweet_score,tweet.retweets)
                        max_like_score=max(max_like_score,tweet.favorites)

                        # maintain tweet with tweet ID in a dictionary
                        tweet_dict[tweetID] = [tweet_text,0,tweet.retweets,tweet.favorites]

                        tweet_tokens_dict[tweetID]=final_tokens

                        #print(final_tokens)
                        final_token_list=list(set(final_token_list+final_tokens))

                        for token in set(final_tokens):
                            tokens_df_dict.setdefault(token,0)
                            tokens_df_dict[token]+=1
                            
                        tweetID += 1
            end = time.time()
        while startID!=tweetID:
            #print(startID,tweet_dict[startID])
            tweet_dict[startID][1]=((0.7*tweet_dict[startID][2]/max_tweet_score)+(0.3*tweet_dict[startID][3]/max_like_score))
            #print(max_tweet_score,max_like_score)
            #print(tweet_dict[startID][1])
            startID+=1
        print("-----------------------------------------")

2019-03-23
@IPL
IPL
-----------------------------------------
2019-03-23
@ESPNcricinfo
IPL
-----------------------------------------
2019-03-23
@cricbuzz
IPL
-----------------------------------------
2019-03-23
@Cricketracker
IPL
-----------------------------------------
2019-03-23
@circleofcricket
IPL
-----------------------------------------
2019-03-24
@IPL
IPL
-----------------------------------------
2019-03-24
@ESPNcricinfo
IPL
-----------------------------------------
2019-03-24
@cricbuzz
IPL
-----------------------------------------
2019-03-24
@Cricketracker
IPL
-----------------------------------------
2019-03-24
@circleofcricket
IPL
-----------------------------------------
2019-03-25
@IPL
IPL
-----------------------------------------
2019-03-25
@ESPNcricinfo
IPL
-----------------------------------------
2019-03-25
@cricbuzz
IPL
-----------------------------------------
2019-03-25
@Cricketracker
IPL
-----------------------------------------
2019-03-25
@circleofcricket
IPL
----

-----------------------------------------
2019-04-16
@IPL
IPL
-----------------------------------------
2019-04-16
@ESPNcricinfo
IPL
-----------------------------------------
2019-04-16
@cricbuzz
IPL
-----------------------------------------
2019-04-16
@Cricketracker
IPL
-----------------------------------------
2019-04-16
@circleofcricket
IPL
-----------------------------------------


In [7]:
vocab_size=len(set(final_token_list))
vocab=sorted(set(final_token_list))

In [8]:
tweet_df=pd.DataFrame(columns=vocab)

for key, val in tweet_tokens_dict.items():
    row=[0 for x in range(vocab_size)]
    final_tokens=Counter(tweet_tokens_dict[key])
    for i,term in enumerate(vocab):
        if term in set(tweet_tokens_dict[key]):
            idf = math.log10(tweetID / tokens_df_dict[term])
            row[i]=final_tokens[term]*idf
    tweet_df = tweet_df.append(pd.Series(row, index=tweet_df.columns), ignore_index=True)

In [9]:
tweet_df.shape

(1407, 2675)

In [10]:
def cosine_similarity(vect1,vect2):
    return np.dot(vect1,vect2)/(np.linalg.norm(vect1)*np.linalg.norm(vect2))

In [11]:
def jaccard_similarity(list1,list2):
    doc_intersection=len(set(list1).intersection(set(list2)))
    return (doc_intersection)/(len(set(list1))+len(set(list2))-doc_intersection)

In [12]:
with open('contextual_vector.pkl','rb') as f:
        contextual_vector_dict = pickle.load(f)

In [13]:
common_terms=['win','won','romp','wins','team','season','game','like','toss','wicket','match','xi','vs','runs','four','six','dot','ball','overs','innings','vivoipl','ipl','balls','run']

In [14]:
with open('model_1_evaluation_data.pkl','rb') as f:
        schedule_dict = pickle.load(f)

In [15]:
for key,val in contextual_vector_dict.items():
    temp=[]
    for v in val:
        if v[0]=="raina🇮🇳":
            temp.append("raina")
        elif v[0] not in common_terms:
            temp.append(v[0])
    if key in list(schedule_dict):
        #print(temp)
        val=schedule_dict[key]
        for v in val:
            if len(v)>1 and v not in common_terms and v not in temp:
                    temp.append(v)
    contextual_vector_dict[key]=temp

In [16]:
for key,val in contextual_vector_dict.items():
    print(key,val)

2019-03-23 ['raina', 'opener', 'dhoni', 'opening', 'champions', 'chepauk', 'kohli', 'chennai', 'super', 'kings', 'beat', 'royal', 'challengers', 'bangalore', 'by', 'wickets', 'table', 'top']
2019-03-24 ['sharma', 'kolkata', 'playing', 'kkrvsrh', 'rishabh', 'bumrah', 'mumbai', 'pant', 'yuvraj', 'singh', 'man', 'indians', 'knight', 'riders', 'beat', 'sunrisers', 'hyderabad', 'by', 'wickets', 'delhi', 'capitals', '37', 'table', 'top']
2019-03-25 ['rajasthan', 'royals', 'jaipur', 'punjab', 'jos', 'kings', 'left', 'first', 'chris', 'beat', 'by', '14', 'table', 'top']
2019-03-26 ['two', 'playing', 'pant', 'rrvkxip', 'capitals', 'bravo', 'delhi', 'super', 'kings', 'chennai', 'beat', 'by', 'wickets', 'table', 'top']
2019-03-27 ['kings', 'top', 'punjab', 'agarwal', 'kolkata', 'andre', 'riders', 'knight', 'russell', 'bowled', 'beat', 'by', '28', 'table']
2019-03-28 ['malinga', 'chahal', 'bumrah', 'kohli', 'virat', 'villiers', 'ab', 'de', 'cricket', 'first', 'mumbai', 'indians', 'beat', 'royal', 

In [17]:
tweet_list=[key for key,val in sorted(dict(filter(lambda x:x[1][1]>=0.1,tweet_dict.items())).items(),key=lambda x: x[1][1],reverse=True)]
tweet_list_copy=[key for key,val in sorted(dict(filter(lambda x:x[1][1]>=0.1,tweet_dict.items())).items(),key=lambda x: x[1][1],reverse=True)]
clus_dict={}
#print(tweet_list)
for tweetID1 in tweet_list:
    if len(tweet_list_copy)==0:
        break
    if tweetID1 in set(tweet_list_copy):
        clus_dict.setdefault(tweetID1,[])
        #print(tweetID1,tweet_dict[tweetID1][0],tweet_dict[tweetID1][1])
        for tweetID2 in tweet_list:
            if tweetID2 in set(tweet_list_copy):
                if tweetID1!=tweetID2:
                    cosine_score=cosine_similarity(tweet_df.loc[tweetID1],tweet_df.loc[tweetID2])
                    #jaccard_score=jaccard_similarity(tweet_tokens_dict[tweetID1],tweet_tokens_dict[tweetID2])
                    if cosine_score>0.1:
                        if(len(clus_dict[tweetID1])<=len(source_list)):
                            clus_dict[tweetID1].append(tweetID2)
                            tweet_list_copy.remove(tweetID2)
        tweet_list_copy.remove(tweetID1)
        #print(clus_dict[tweetID1])
        #print(tweet_list)
        #print("---------------------------------------------")

In [18]:
#tweet_list=[key for key,val in sorted(dict(filter(lambda x:x[1][1]>=0.5,tweet_dict.items())).items(),key=lambda x: x[1][1],reverse=True)]
index=0
clus_dict_copy={}
tweet_timeline_2_dict={}
for key,val in clus_dict.items():
    clus_dict_copy[key]=val
temp=[]
for key,val in contextual_vector_dict.items():
    #print(val)
    index=index+1
    #if index!=len(list(contextual_vector_dict)):
    if index>25:
          break
    contextual_vector=[1 for x in range(len(val))]
    max_score=0
    tweet_list=list(clus_dict_copy)
    best_tweet_ID=tweet_list[0]
    for tweetID in set(tweet_list):
        vector=[0 for x in range(len(val))]
        tokens_list=tweet_tokens_dict[tweetID]
        if "raina🇮🇳" in tokens_list:
            tokens_list[tokens_list.index("raina🇮🇳")]="raina"
        for i,v in enumerate(val):
            if v in set(tokens_list):
                vector[i]=1
        cosine_score=0
        total_score=0
        if sum(vector)>2:
            cosine_score=cosine_similarity(contextual_vector,vector) 
            total_score=cosine_score#+0.5*tweet_dict[tweetID][1]
            #if cosine_score>=0.3:
        
        '''
        print(val)
        print(tweet_tokens_dict[tweetID])
        print(vector)
        print("Cosine score:",cosine_score)
        print("Tweet quality score:",tweet_dict[tweetID][1])
        print(tweet_dict[tweetID][0])
        print(total_score)
        print("---------------------------")
        '''

        if max_score<total_score:
            max_score=total_score
            best_tweet_ID=tweetID
            temp=vector
    del clus_dict_copy[best_tweet_ID]
    print(key,tweet_dict[best_tweet_ID][0],val,temp,max_score,tweet_dict[best_tweet_ID][1])
    tweet_timeline_2_dict[key]=tweet_dict[best_tweet_ID][0]
    print("---------------------------")
    #print(val)
    #print(tweet_tokens_dict[best_tweet_ID])

2019-03-23  #ChinnaThala @ImRaina wins the race against Virat Kohli to become the first batsman to scale 5000 #VIVOIPL runs #CSKvRCB ['raina', 'opener', 'dhoni', 'opening', 'champions', 'chepauk', 'kohli', 'chennai', 'super', 'kings', 'beat', 'royal', 'challengers', 'bangalore', 'by', 'wickets', 'table', 'top'] [1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0] 0.6666666666666666 1.0
---------------------------
2019-03-24 Youngest to take five-fers in IPL: #SRHvsMI #IPL2019 21y 204d J Unadkat RCB v DD Delhi 2013 22y 137d A JOSEPH MI v SRH Hyderabad 2019 * 22y 237d I Sharma DC v KTK Kochi 2011 ['sharma', 'kolkata', 'playing', 'kkrvsrh', 'rishabh', 'bumrah', 'mumbai', 'pant', 'yuvraj', 'singh', 'man', 'indians', 'knight', 'riders', 'beat', 'sunrisers', 'hyderabad', 'by', 'wickets', 'delhi', 'capitals', '37', 'table', 'top'] [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0] 0.5400617248673216 0.8722901737508589
---------------------------
2019-03-25 BIG. BIG. CO

In [712]:
f = open("tweet_timeline_2.pkl", "wb")
pickle.dump(tweet_timeline_2_dict, f)
f.close()

In [714]:
tweet_df.to_csv("model_2_vector.csv")

In [715]:
f = open("tweet_dict_model_2.pkl", "wb")
pickle.dump(tweet_dict, f)
f.close()

In [716]:
f = open("tweet_tokens_dict_model_2.pkl", "wb")
pickle.dump(tweet_tokens_dict, f)
f.close()

In [19]:
f = open("cluster_dict_model_2.pkl", "wb")
pickle.dump(clus_dict, f)
f.close()

In [717]:
i=1
for key,val in clus_dict.items():
    if len(val)>=len(source_list):
        print("Cluster {}".format(i))
        print(tweet_dict[key][0],tweet_dict[key][1])
        for v in val:
            print(tweet_dict[v][0],tweet_dict[v][1])
        i=i+1
        print()


Cluster 1
 #ChinnaThala @ImRaina wins the race against Virat Kohli to become the first batsman to scale 5000 #VIVOIPL runs #CSKvRCB 1.0
Congratulations to Suresh Raina for becoming the first batsman to score 5,000 runs in the IPL. #CSKvRCB #IPL2019 1.0
Quickest to 4000 runs in the IPL:  @henrygayle @davidwarner31 @imVkohli @GautamGambhir @ImRaina @SDhawan25 @ImRo45 @robbieuthappa @msdhoni 1.0
Virat Kohli becomes only the second batsman to score 5,000 IPL runs; he is the first to reach the milestone for a single franchise. #RCBvMI #IPL2019 1.0
We can not see @imVkohli like this. #RCBvDC #IPL2019 #IPL12 #ViratKohli 1.0
A victory at Mohali as @RCBTweets register their first win of the #VIVOIPL 2019 season  1.0
it is good to see Virat Kohli as the winning captain. #IPL2019 1.0

Cluster 2
Words of wisdom from none other than the Master Blaster for young @PrithviShaw at Wankhede. #VIVOIPL 1.0
Take a bow @KagisoRabada25. What an over from the young lad as the @DelhiCapitals defend 10 runs in 

Not a record RCB would be boastful about. #CSKvRCB #IPL2019 0.20257838433036754
Maiden IPL hundred for Jonny Bairstow. #SRHvRCB #IPL 0.15913563332918168
Alzarri Joseph created a new record on his IPL debut. #SRHvMI 0.12176417776447701
Here are the best tweets on Bairstow's ton. #IPL #SRHvRCB 0.10772823514758999

Cluster 96
Here's how cricketers reacted to Rishabh Pant's knock. #IPL #IPL2019 #MIvDC 0.32235294117647056
Should wives and girlfriends be allowed with cricketers on tours? @DanishSait asked this week's #StrangerXI and like always, there were a few surprises. #IPL2019 #IndiaCricket 0.23843746238112434
A stunning comeback from @KKRiders to get up to 185. Batting through pain @Russell12A powers his team with a superb knock of 62. #VIVOIPL 0.23819329656692362
Andre Russell departs after playing an exhilarating knock of 62(28) - Morris gets him! #KKR - 156/6 after 18 .. #DCvKKR #IPL2019  0.1610689761197089
SIX! Pant gets down on one knee and hits it over long on to bring up the -ru

In [42]:
import numpy as np
vect=[]
for key,val in tweet_tokens_dict.items(): 
    vect=list(set(vect+val))
vect=np.array(vect).reshape(-1,1)
vect

array([['square'],
       ['enjoys'],
       ['kolkataknightriders'],
       ...,
       ['cramped'],
       ['hyderabad'],
       ['comfortably']], dtype='<U19')

In [43]:
import gensim
from gensim.models import Word2Vec
model = Word2Vec(vect,min_count=1)

In [34]:
model.wv.vocab

{'hussey': <gensim.models.keyedvectors.Vocab at 0x1d68b69b710>,
 'square': <gensim.models.keyedvectors.Vocab at 0x1d68b4d0ef0>,
 'enjoys': <gensim.models.keyedvectors.Vocab at 0x1d688d01550>,
 'kolkataknightriders': <gensim.models.keyedvectors.Vocab at 0x1d688d011d0>,
 'upton': <gensim.models.keyedvectors.Vocab at 0x1d68b539240>,
 '33ball': <gensim.models.keyedvectors.Vocab at 0x1d68b539278>,
 'stuart': <gensim.models.keyedvectors.Vocab at 0x1d68b5392b0>,
 'return': <gensim.models.keyedvectors.Vocab at 0x1d68b74e128>,
 'note': <gensim.models.keyedvectors.Vocab at 0x1d68b74e198>,
 'losing': <gensim.models.keyedvectors.Vocab at 0x1d68b74e1d0>,
 'eyes': <gensim.models.keyedvectors.Vocab at 0x1d68b74e208>,
 'receives': <gensim.models.keyedvectors.Vocab at 0x1d68b74e240>,
 'dramatic': <gensim.models.keyedvectors.Vocab at 0x1d68b74e278>,
 'robbieuthappa': <gensim.models.keyedvectors.Vocab at 0x1d68b74e2b0>,
 'bhuvneshwar': <gensim.models.keyedvectors.Vocab at 0x1d68b74e2e8>,
 'update': <gens

In [44]:
tweet_wv={}
for key,val in tweet_tokens_dict.items():
    words = [word for word in val if word in model.wv.vocab]
    tweet_wv[key]=np.mean(model.wv[words], axis=0)

In [45]:
from nltk.cluster import KMeansClusterer
import nltk
NUM_CLUSTERS=13
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.euclidean_distance,avoid_empty_clusters=True)
assigned_clusters = kclusterer.cluster(tweet_wv.values(), assign_clusters=True)
print (assigned_clusters)

[12, 1, 7, 5, 1, 1, 1, 1, 3, 7, 7, 1, 2, 3, 10, 1, 1, 9, 12, 12, 3, 1, 7, 12, 3, 12, 1, 1, 1, 12, 10, 12, 7, 12, 5, 12, 12, 2, 2, 7, 1, 1, 7, 1, 1, 1, 12, 12, 12, 7, 1, 1, 1, 7, 9, 2, 8, 1, 3, 1, 1, 2, 1, 3, 8, 3, 1, 2, 1, 3, 4, 2, 8, 3, 8, 2, 2, 0, 0, 7, 0, 6, 7, 8, 2, 0, 8, 1, 1, 1, 3, 1, 1, 4, 0, 10, 0, 4, 6, 1, 3, 1, 2, 3, 1, 0, 1, 1, 4, 1, 10, 1, 4, 0, 0, 4, 1, 1, 0, 6, 4, 0, 1, 1, 10, 0, 0, 2, 2, 7, 1, 1, 3, 2, 1, 3, 6, 3, 9, 3, 12, 3, 5, 3, 3, 3, 10, 3, 1, 1, 10, 12, 12, 3, 3, 3, 3, 8, 3, 1, 1, 12, 2, 12, 10, 6, 10, 1, 10, 0, 10, 2, 4, 0, 0, 3, 3, 3, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 10, 1, 3, 9, 7, 10, 3, 6, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 10, 12, 12, 5, 1, 1, 6, 6, 6, 6, 1, 3, 7, 1, 1, 0, 3, 7, 1, 1, 1, 3, 5, 1, 10, 1, 1, 1, 1, 0, 1, 1, 0, 8, 0, 0, 1, 1, 1, 1, 1, 2, 0, 8, 8, 0, 1, 6, 1, 0, 10, 10, 5, 10, 1, 1, 1, 1, 8, 1, 8, 1, 0, 1, 8, 3, 3, 1, 6, 1, 6, 1, 3, 1, 3, 7, 8, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 6, 7, 6, 1, 6, 4, 2, 1, 2, 10, 3, 1, 0, 9, 1, 1, 1, 12, 1, 6, 1, 1,

In [46]:
final_clusters_dict={}
for tweetID,tweet in tweet_dict.items():  
    if tweet[1]>0.8:
        final_clusters_dict.setdefault(assigned_clusters[tweetID],[]).append((tweet[0],tweet[1]))
final_clusters_dict={key:val for key,val in sorted(final_clusters_dict.items(), key=lambda x: x[0])}

In [39]:
for key, val in final_clusters_dict.items():
    print("Cluster {}".format(key))
    for v in val:
        print(v[0],v[1])
    print()

Cluster 0
SAM CURRAN -- HAT-TRICK !! : Harshal Patel c Rahul b Sam Curran 0(2) : Rabada b Sam Curran 0(1) : Lamichhane b Sam Curran 0(1) #IPL2019 #KXIPvDC 0.9572966507177033

Cluster 2
”I would love him to carry on for Chennai till the end of this cycle and we will then assess. But it is a question for him and his motivation,” @SPFleming7 hopes @msdhoni will continue playing for @ChennaiIPL. #IPL2019 1.0
Chennai Super Kings record their fifth successive win in #IPL.  #IPL2019 #DCvCSK 1.0
Gracia Raina and Ziva Dhoni reunite during the #DCvCSK match in Delhi #CSK #IPL2019 1.0
.@ChennaiIPL remain the only unbeaten team in #VIVOIPL 2019 and go top of the table #CSKvRR 0.8046575624920694
What a thriller we have witnessed at the Chepauk! @ChennaiIPL beat RR by 8 runs #CSKvRR #VIVOIPL 0.970439189189189
#CSK have beaten #RR by 8 runs to make it 3 wins out of 3!! All hail the captain of captains, MS Dhoni!! WHAT A FINISH! #CSKvRR #IPL2019 1.0

Cluster 3
A look at the points table after today's 